
## **0. はじめに**

日本語の処理もまじえながら、コンペデータを使いつつ自分用にNLPを少しずつまとめていきます。<br>
ゼロから作るDeep Learning―自然言語処理編で理解したこともまとめていきます。<br>
RNNからまとめていきますが、少しずつ書き足していきます。<br>

#### 0.1　最新版の日本語対応transformer インストール

現在のtransformersはmecabではなくfugashiを使っています。（fugashiはMeCabのラッパー）<br>
以下を実行するだけで、fugashiも辞書も全部入ります。<br>
昔はfugashiを別途installしていた。<br>

https://qiita.com/m__k/items/863013dbe847dc613844

In [ ]:
!pip install transformers[ja]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
to
import gensim.downloader
import transformers
from wordcloud import WordCloud

train = pd.read_csv('../input/nlp-getting-started/train.csv',dtype={'id': np.int16, 'target': np.int8})

## Table of Contents:<br>
1. [refernce](https://www.kaggle.com/tfukuda675/lnp-beginner#1.-reference)<br>
2. [Unique process of NLP](https://www.kaggle.com/tfukuda675/lnp-beginner#2.-Unique-process-of-NLP)<br>
 2.1 Tokenization
 2.2 Padding
3. [RNN](https://www.kaggle.com/tfukuda675/lnp-beginner#3.-RNN)<br>
 3.1 simple code<br>
4. LSTM<br>
 4.1 simple code<br>
5. Transformer<br>
 5.1 base line<br>

## **1. reference**

#### This kernel includes codes and ideas from kernels below.
* [NLP with Disaster Tweets - EDA, Cleaning and BERT](https://www.kaggle.com/tfukuda675/nlp-with-disaster-tweets-eda-cleaning-and-bert/edit/run/78390965) by [gunesevitan](https://www.kaggle.com/gunesevitan)
* [n-Depth Guide 📙 to Google's BERT](https://www.kaggle.com/ratan123/in-depth-guide-to-google-s-bert) by [ratan rohith]
(https://www.kaggle.com/ratan123)
* [Introduction to Japanese spaCy/GINZA [日本語/Eng]](https://www.kaggle.com/marutama/introduction-to-japanese-spacy-ginza-eng) by [NIWASHI](https://www.kaggle.com/marutama)
* <a href="https://jp.mathworks.com/content/dam/mathworks/mathworks-dot-com/company/events/webinar-cta/2514077_JP_2018-09-07_Deep_Learning_LSTM_1.pdf#page=9">Structure of RNN</a>
* [深層学習／ゼロから作るDeep Learning２　第５章メモ](https://qiita.com/jun40vn/items/35f6f0d26f9e58f01e4e)
* [深層学習／ゼロから作るDeep Learning２　第６章メモ](https://qiita.com/jun40vn/items/e690dfe80faa6512049f)
* [自然言語処理の必須知識 Transformer を徹底解説！](https://deepsquare.jp/2020/07/transformer/)
* [LSTMネットワークの概要](https://qiita.com/KojiOhki/items/89cd7b69a8a6239d67ca)
* [colah氏の解説 Understanding LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [画像用Transformerを利用して衛星画像の分類機械学習モデルを作成する](https://sorabatake.jp/20454/) @空畑
* [PyTorchで自然言語処理でよく使用されるTransformerを少ないコードで使用してみる](https://www.yurui-deep-learning.com/2021/01/07/pytorch-transformer/)
* [LSTMによる系列データの予測と分類](https://jp.mathworks.com/content/dam/mathworks/mathworks-dot-com/company/events/webinar-cta/2514077_JP_2018-09-07_Deep_Learning_LSTM_1.pdf) @mathworks
* [RNNの問題点](https://www.anarchive-beta.com/entry/2021/01/07/180000)
* [作って理解する Transformer / Attention](https://qiita.com/halhorn/items/c91497522be27bde17ce)
* [AI界を席巻する「Transformer」をゆっくり解説](https://zenn.dev/attentionplease/articles/1a01887b783494)
* [論文解説 Attention Is All You Need (Transformer)](https://deeplearning.hatenablog.com/entry/transformer)
* [自然言語処理の巨獣「Transformer」のSelf-Attention Layer紹介](https://medium.com/lsc-psd/%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86%E3%81%AE%E5%B7%A8%E7%8D%A3-transformer-%E3%81%AEself-attention-layer%E7%B4%B9%E4%BB%8B-a04dc999efc5)
* [Transformers Explained Visually (Part 3): Multi-head Attention, deep dive](https://towardsdatascience.com/transformers-explained-visually-part-3-multi-head-attention-deep-dive-1c1ff1024853)
* [はじめての自然言語処理](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/)

<br><br>
#### 各Layerの説明
* [positional encoding](https://qiita.com/halhorn/items/c91497522be27bde17ce#positional-encoding)
* [Feed Forward Net](https://www.hellocybernetics.tech/entry/2016/05/22/014656)
  隠れ層1層の全結合NN
* [Batch Normalize](https://qiita.com/t-tkd3a/items/14950dbf55f7a3095600#%E5%85%A8%E7%B5%90%E5%90%88nn-%E3%81%AE-batch-normalization)
* [Layer Normalization](https://data-analytics.fun/2020/07/16/understanding-layer-normalization/)
* [gMLP](https://deepsquare.jp/2021/05/gmlp/)

<br><br>
#### よく忘れる単語まとめ
* [ノルム](https://manabitimes.jp/math/1269)
* [アマダール積](https://python.atelierkobato.com/hadamard/)<br>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/0866ce505a5888cd4db8e6ba2b710fe8cd5a6578" width="500">
* [アフィン変換](https://qiita.com/koshian2/items/c133e2e10c261b8646bf)<br>
全結合層による変換の事。幾何学の分野ではアフィン変換に相当するので、ここではアフィン変換と読んでいる。<br>
* tanhとsigmoidについて<br>
tanhは、何かしらの情報の出力についている。情報に対する強弱と解釈できる。<br>
sigmoidは、データをどれだけ通すかという割合。<br>
* [Batch Normalization](https://yaakublog.com/batch-normalization)<br>
* [評価関数まとめ](https://amateur-engineer-blog.com/machine-learning-metrics/)<br>


## **2. Unique process of NLP**

RNN, LSTM, transformer cannot treat texts directory.<br>
At first, we need to run tokernize. This process transform words to ids.<br>
Next, run padding process. this process add special id to align the length to the longest sentence.<br>
Those process use language model, BERT etc,.<br>
<br>
文字列をそのまま処理できない為、まずトークン化を行います。<br>
その後、各センテンスの長さを揃える為、paddingを行います。<br>
これらのプロセスでは、BERTで知られるようなモデルを使います。<br>
当然モデルの影響も大きいです。<br>
モデルについては別途まとめます。<br>

#### 2.1 Tokenization
文字列をそのまま学習する事はできません。<br>
学習できるように、数値に置き換える処理が必要です。それをトークン化(Tokenize)と言います。<br>
以下の画像の上2つがトークン化に相当します。<br>
<img src="https://camo.qiitausercontent.com/51b2ff47f2bb952f38a63b02d056ebfc7dea097d/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f36323435302f35323261393063362d613732352d333731362d346161382d6637626563396430396262622e706e67" width=600>
<br>

#### 2.2 Tokenization result of 1st text

Show BERT tokenizer result with transformer bert model.<br>
The 'input_ids' result is tokenization text.<br>
<br>
日本語を扱う場合は、日本語のtokenizerを利用する。<br>

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-cased')
##== whwn you want to treat Japanese character,
# !pip install transformers[ja]
#tokenizer = transformers.AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
tokenizer(train.text[0])

#### 2.2 Padding
学習時には入力は固定長が期待されています。<br>
トークン化したものの長さを揃えるため、paddingで"0"などを追加します。<br>
<br>

padding前のトークン化済みデータ
```
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]
```
padding後
```
## Result
[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]
```
<br>

## **3. RNN** 

#### 3.1 RNN one layer

Understand rnn structure via python code.<br>

#### 式の説明
##### **forwarについて**<br>
<img src="https://camo.qiitausercontent.com/d0046c189cf724199009651c48433d0097df591a/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f34303634653037662d623962622d303432392d616339322d6338366233313837616164622e706e67" width=600>
<br><br>

##### **backwardについて**<br>

<img src="https://camo.qiitausercontent.com/8942bf9b069aece2d1c19a676a089414643a02c8/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f31396133383030642d336236342d393133652d396165342d3731616664653534356138652e706e67" width=600>

<br><br>
#### **コードの説明**
three dots については[こちら](https://note.nkmk.me/python-numpy-ellipsis/)<br>

In [ ]:
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
    
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, d = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(x.T, dt)
        dx = np.dot(dt, Wx.T)
        
        self.grads[0][...] = dWx  ## three dots
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev

#### 3.２ Time RNN one layer

RNNを横にT個並べたもの。<br>
RNNで一つのセンテンス長を取り扱い、横にT個並べる事で時間方向の解像度を作り出している。<br>

#### 式の説明
<br>

##### **forwardについて**
<br>
<img src="https://camo.qiitausercontent.com/3d6925879a4d36d8cd7562a9adf75d76f43f281e/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f65363761373333612d623134662d373232362d646262382d3061393135396663396434312e706e67" width="600">
<br><br>

##### **backwordについて**
<br>
<img src="https://camo.qiitausercontent.com/78da7c43ce5218b6eb574f29e3ffb574f977b92e/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f39383436336136652d313534392d356264322d373530312d6635353066346538303932302e706e67" width="400">
<br><br>


In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = RNN(*self.params)  ## <= RNNをT個生成している。
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs
    
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh)
            dxs[:, t, :] = dx

            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None

#### **3.3 RNNLM**

TimeRNN　layerに加え、TimeAffine　layer、 TimeEmbedding layer、TimeSoftmaxWithLoss　layerを組み合わせて<br>
RNNLMを作る。


##### **Structure of RNNLM**

<img src ="https://camo.qiitausercontent.com/ac8303dfae114898ac9d1f752e0b9378ba975784/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f61346461333038372d623736642d353636322d386539322d3364306131333062643737342e706e67" width="500">
<br><br>

##### **Time Embedding**

<img src="https://camo.qiitausercontent.com/32654fed87f53f2b82b773d50c36e94a97f9bec0/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f31363033633765302d653532392d346237632d653936312d3039396561653661663433632e706e67" width="500">
<br>
Time Embeddingレイヤは、xsから1列づつデータを切り出しEmbeddingレイヤに入力して、<br>
その出力をout(N, T, D)に溜め込む、という動作をforループでT回繰り返すというものです。
<br><br>

##### **Time Affine Layer**

<img src="https://camo.qiitausercontent.com/40771ddfdce111906aa55c50a000af060e146803/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f37393361623630622d336263332d653063332d663166342d3064316436653038333730352e706e67" width="500">
<br>
Time Affineレイヤは、Affineレイヤの入出力に、時間軸方向のTに対応出来るようにそれぞれreshapeを付け加えたものです。<br>
<br><br>

##### **Time SoftMax WithLoss**
<img src="https://camo.qiitausercontent.com/e2d2604218ab475a245c7bae0a160e7f97416108/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f63616563366363642d373362332d663733612d653230352d3737363838326639346230652e706e67" width="400">
<br>
Time Softmax with Loss レイヤー は、xt,ttxt,ttのSotmax with Loss をT個合算してTで割るレイヤーです。<br>

#### **コードの説明**
three dots については[こちら](https://note.nkmk.me/python-numpy-ellipsis/)<br>

In [ ]:
class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        # 重みの初期化
        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        # レイヤの生成
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

        # すべての重みと勾配をリストにまとめる
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.rnn_layer.reset_state()
        
class TimeEmbedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W

    def forward(self, xs):
        N, T = xs.shape
        V, D = self.W.shape

        out = np.empty((N, T, D), dtype='f')
        self.layers = []

        for t in range(T):
            layer = Embedding(self.W)
            out[:, t, :] = layer.forward(xs[:, t])
            self.layers.append(layer)

        return out

    def backward(self, dout):
        N, T, D = dout.shape

        grad = 0
        for t in range(T):
            layer = self.layers[t]
            layer.backward(dout[:, t, :])
            grad += layer.grads[0]

        self.grads[0][...] = grad
        return None

class TimeAffine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        N, T, D = x.shape
        W, b = self.params

        rx = x.reshape(N*T, -1)
        out = np.dot(rx, W) + b
        self.x = x
        return out.reshape(N, T, -1)

    def backward(self, dout):
        x = self.x
        N, T, D = x.shape
        W, b = self.params

        dout = dout.reshape(N*T, -1)
        rx = x.reshape(N*T, -1)

        db = np.sum(dout, axis=0)
        dW = np.dot(rx.T, dout)
        dx = np.dot(dout, W.T)
        dx = dx.reshape(*x.shape)

        self.grads[0][...] = dW
        self.grads[1][...] = db

        return dx
    
class TimeSoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None
        self.ignore_label = -1

    def forward(self, xs, ts):
        N, T, V = xs.shape

        if ts.ndim == 3:  # 教師ラベルがone-hotベクトルの場合
            ts = ts.argmax(axis=2)

        mask = (ts != self.ignore_label)

        # バッチ分と時系列分をまとめる（reshape）
        xs = xs.reshape(N * T, V)
        ts = ts.reshape(N * T)
        mask = mask.reshape(N * T)

        ys = softmax(xs)
        ls = np.log(ys[np.arange(N * T), ts])
        ls *= mask  # ignore_labelに該当するデータは損失を0にする
        loss = -np.sum(ls)
        loss /= mask.sum()

        self.cache = (ts, ys, mask, (N, T, V))
        return loss

    def backward(self, dout=1):
        ts, ys, mask, (N, T, V) = self.cache

        dx = ys
        dx[np.arange(N * T), ts] -= 1
        dx *= dout
        dx /= mask.sum()
        dx *= mask[:, np.newaxis]  # ignore_labelに該当するデータは勾配を0にする

        dx = dx.reshape((N, T, V))

        return dx

## **4. LSTM**

1つのLSTMの中に、forget agteと、input gateとstateを記憶するcellと、outputゲートが<br>
入力側から並列に、それぞれのoutputが直列につながっている。<br>
この構造がわかれば、それぞれのパーツは簡単。<br>

#### 4.1.1 Confirm "gradient vanishing problem"

<br>

#### 4.1.2 Confirm "gradient exploding problem"

<br>

#### 4.1.3 diff RNN and LSTM

* RNN<br>
<img src="https://camo.qiitausercontent.com/b643944d722e601f9e3d4b7856cd096895f5ce1f/687474703a2f2f636f6c61682e6769746875622e696f2f706f7374732f323031352d30382d556e6465727374616e64696e672d4c53544d732f696d672f4c53544d332d53696d706c65524e4e2e706e67" width=500>
<br><br>
* LSTM<br>
<img src="https://camo.qiitausercontent.com/a12fe62032a6633b05b8ef7c2512d1e54c9c4afd/687474703a2f2f636f6c61682e6769746875622e696f2f706f7374732f323031352d30382d556e6465727374616e64696e672d4c53544d732f696d672f4c53544d332d636861696e2e706e67" width=500><br><br>

<br>

#### ** 4.1.4 LSTM Core Idea
The key to LSTMs is the cell state, the horizontal line running through the top of the diagram.<br>
<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-C-line.png" width=500><br><br>

#### **コードの説明**

Dx4Hで、4がついている理由は、fとgとiとoのWを横につなげて一気に扱っている為。


#### ４.2 LSTM one layer

Understand LSTM structure via python code.<br>

#### 式の説明
##### **forwarについて** <br>
<img src="https://camo.qiitausercontent.com/b9387ed37d1f247f8f679184f8dd2efd1e7a709a/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f35306164343264652d643433642d333162392d333761652d3637666462313032393939612e706e67" width=600>
<br><br>

##### **backwardについて** <br>
<img src="https://camo.qiitausercontent.com/bd2fad9749a2dd322a1dd9f8dcfbcf3ee209107a/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f62366631386438372d613436662d643762302d303435382d3733656163373434663465662e706e67" width=600>
<br><br>


In [ ]:
class LSTM:
    def __init__(self, Wx, Wh, b):

        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev, c_prev):
        Wx, Wh, b = self.params
        N, H = h_prev.shape

        A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b

        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]

        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)

        c_next = f * c_prev + g * i
        h_next = o * np.tanh(c_next)

        self.cache = (x, h_prev, c_prev, i, f, g, o, c_next)
        return h_next, c_next

    def backward(self, dh_next, dc_next):
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, c_next = self.cache

        tanh_c_next = np.tanh(c_next)

        ds = dc_next + (dh_next * o) * (1 - tanh_c_next ** 2)

        dc_prev = ds * f

        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i

        di *= i * (1 - i)
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2)

        dA = np.hstack((df, dg, di, do))

        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)

        return dx, dh_prev, dc_prev

## **5. Transformer**


## **6. Visualization**

#### **6.1 word cloud**

In [ ]:
words = [ w for t in train["text"].str.split().tolist() for w in t]

In [ ]:
plt.figure(figsize=(12,8))
word_cloud = WordCloud(
                          background_color='black',
                          max_font_size = 80
                         ).generate(" ".join(words[:200]))
plt.imshow(word_cloud)
plt.axis('off')
plt.show()